In [1]:
from pathlib import Path
import pandas as pd 

path = Path("../Baseline") / "df_baseline.csv"
df_baseline = pd.read_csv(path)

nodes - узлы (аккаунты + география)

In [3]:
# Группируем по аккаунту и берём первую локацию
sender_locations = (
    df_baseline[['Sender_account', 'Sender_bank_location']]
    .dropna()
    .groupby('Sender_account')
    .agg(lambda x: x.value_counts().index[0])  # наиболее частая локация
)

receiver_locations = (
    df_baseline[['Receiver_account', 'Receiver_bank_location']]
    .dropna()
    .groupby('Receiver_account')
    .agg(lambda x: x.value_counts().index[0])
)

# Создаём таблицу узлов
all_accounts = pd.unique(df_baseline[['Sender_account', 'Receiver_account']].values.ravel())
df_nodes = pd.DataFrame({'id': all_accounts})

# Маппим локации
df_nodes['location'] = df_nodes['id'].map(sender_locations['Sender_bank_location'])
df_nodes['location'] = df_nodes['location'].fillna(df_nodes['id'].map(receiver_locations['Receiver_bank_location']))

# Сохраняем в CSV
df_nodes.to_csv('nodes.csv', index=False)


edges - ребра (характеристики транзакций)

In [ ]:
df_edges = df_baseline[[
    'Sender_account', 'Receiver_account', 'Amount', 
    'Payment_type', 'Is_laundering', 'month', 'weekday', 'hour', 
    'Payment_currency_iso', 'Received_currency_iso'
]].copy()

# Переименуем столбцы, чтобы соответствовать Cypher
df_edges.rename(columns={
    'Sender_account': 'sender',
    'Receiver_account': 'receiver',
    'Payment_type': 'payment_type',
    'Is_laundering': 'suspicious',
    'Payment_currency_iso': 'payment_currency',
    'Received_currency_iso': 'received_currency'
}, inplace=True)

# Преобразуем флаг подозрительности в строку ('true'/'false') — для Neo4j
df_edges['suspicious'] = df_edges['suspicious'].astype(bool).astype(str).str.lower()

# Сохраняем
df_edges.to_csv('edges.csv', index=False)